# 🩺 Medical NER Model Training

**Fine-tune spaCy for Medical Named Entity Recognition**

This notebook trains a custom NER model to extract:
- **SYMPTOM**: Pain, discomfort, fatigue, etc.
- **TREATMENT**: Medications, procedures, therapies
- **DIAGNOSIS**: Medical conditions, diseases
- **BODY_PART**: Anatomical locations
- **PROGNOSIS**: Expected outcomes

**Author:** Himanshu Sharma  
**For:** Emitrr AI Engineer Intern Assignment

---

## 1. Setup & Installation

In [ ]:
# Install dependencies
!pip install -q spacy==3.7.4
!python -m spacy download en_core_web_sm

import spacy
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.util import minibatch, compounding
import json
import random
from pathlib import Path

print(f"spaCy version: {spacy.__version__}")
print("✅ Setup complete!")

## 2. Load Training Data

In [ ]:
# Medical NER Training Data
# Each entry: [text, {"entities": [(start, end, label), ...]}]

TRAINING_DATA = [
    ("I had pain in my neck and back almost right away.", 
     {"entities": [(7, 11, "SYMPTOM"), (18, 22, "BODY_PART"), (27, 31, "BODY_PART")]}),
    
    ("They said it was a whiplash injury, but they didn't do any X-rays.",
     {"entities": [(19, 34, "DIAGNOSIS")]}),
    
    ("I had to take painkillers regularly and go through ten sessions of physiotherapy.",
     {"entities": [(14, 25, "TREATMENT"), (67, 80, "TREATMENT")]}),
    
    ("I do get occasional backaches.",
     {"entities": [(20, 29, "SYMPTOM")]}),
    
    ("Your neck and back have a full range of movement.",
     {"entities": [(5, 9, "BODY_PART"), (14, 18, "BODY_PART")]}),
    
    ("I've been having these terrible headaches for the past two weeks.",
     {"entities": [(23, 41, "SYMPTOM")]}),
    
    ("They start behind my eyes and spread to the back of my head.",
     {"entities": [(20, 24, "BODY_PART"), (49, 53, "BODY_PART")]}),
    
    ("This sounds like tension headaches, which are very common.",
     {"entities": [(17, 34, "DIAGNOSIS")]}),
    
    ("Your blood pressure is elevated at 150/95.",
     {"entities": [(5, 19, "SYMPTOM")]}),
    
    ("Your blood glucose levels indicate pre-diabetes.",
     {"entities": [(35, 47, "DIAGNOSIS")]}),
    
    ("Your X-ray shows mild osteoarthritis in your right knee.",
     {"entities": [(17, 37, "DIAGNOSIS"), (50, 54, "BODY_PART")]}),
    
    ("I'm prescribing lisinopril 10mg daily for your blood pressure.",
     {"entities": [(16, 37, "TREATMENT")]}),
    
    ("For cholesterol, I'm adding atorvastatin 20mg at bedtime.",
     {"entities": [(28, 45, "TREATMENT")]}),
    
    ("We'll try physical therapy first - twice weekly for six weeks.",
     {"entities": [(10, 26, "TREATMENT")]}),
    
    ("I've been having more asthma attacks lately, especially at night.",
     {"entities": [(22, 36, "SYMPTOM")]}),
    
    ("I wake up coughing and can't breathe properly.",
     {"entities": [(10, 18, "SYMPTOM"), (23, 45, "SYMPTOM")]}),
    
    ("I can hear some wheezing in both lungs.",
     {"entities": [(16, 24, "SYMPTOM"), (33, 38, "BODY_PART")]}),
    
    ("I'm going to add a daily inhaled corticosteroid - fluticasone.",
     {"entities": [(25, 48, "TREATMENT"), (51, 62, "TREATMENT")]}),
    
    ("I just have no energy. I feel exhausted.",
     {"entities": [(12, 21, "SYMPTOM"), (30, 39, "SYMPTOM")]}),
    
    ("I've also noticed I'm always cold.",
     {"entities": [(26, 34, "SYMPTOM")]}),
    
    ("I've gained about 8 pounds without changing my diet.",
     {"entities": [(4, 27, "SYMPTOM")]}),
    
    ("My hair seems thinner and my skin is really dry.",
     {"entities": [(3, 21, "SYMPTOM"), (30, 47, "SYMPTOM"), (3, 7, "BODY_PART"), (30, 34, "BODY_PART")]}),
    
    ("This confirms hypothyroidism.",
     {"entities": [(15, 29, "DIAGNOSIS")]}),
    
    ("I'm starting you on levothyroxine 50 micrograms daily.",
     {"entities": [(20, 53, "TREATMENT")]}),
    
    ("I've had this burning feeling in my chest, especially after eating.",
     {"entities": [(14, 41, "SYMPTOM"), (35, 40, "BODY_PART")]}),
    
    ("You likely have gastroesophageal reflux disease, or GERD.",
     {"entities": [(16, 47, "DIAGNOSIS"), (52, 56, "DIAGNOSIS")]}),
    
    ("I'm going to prescribe omeprazole 20mg daily.",
     {"entities": [(23, 40, "TREATMENT")]}),
    
    ("My hands and wrists hurt a lot, especially in the morning.",
     {"entities": [(3, 8, "BODY_PART"), (13, 19, "BODY_PART"), (20, 24, "SYMPTOM")]}),
    
    ("The morning stiffness lasts about an hour or more.",
     {"entities": [(4, 21, "SYMPTOM")]}),
    
    ("I'm concerned this could be rheumatoid arthritis.",
     {"entities": [(28, 48, "DIAGNOSIS")]}),
    
    ("He's been having diarrhea for about a week now. And he throws up sometimes.",
     {"entities": [(17, 25, "SYMPTOM"), (56, 65, "SYMPTOM")]}),
    
    ("He had a fever, it was 101°F yesterday.",
     {"entities": [(9, 14, "SYMPTOM")]}),
    
    ("This looks like a viral gastroenteritis.",
     {"entities": [(18, 39, "DIAGNOSIS")]}),
    
    ("Push clear fluids - small sips frequently. Pedialyte is great.",
     {"entities": [(5, 17, "TREATMENT"), (43, 52, "TREATMENT")]}),
    
    ("You can use acetaminophen for fever.",
     {"entities": [(12, 25, "TREATMENT"), (30, 35, "SYMPTOM")]}),
    
    ("There's a small mass in your left breast.",
     {"entities": [(12, 22, "SYMPTOM"), (31, 42, "BODY_PART")]}),
    
    ("The room just starts spinning out of nowhere.",
     {"entities": [(20, 28, "SYMPTOM")]}),
    
    ("I feel nauseous during the episodes.",
     {"entities": [(7, 15, "SYMPTOM")]}),
    
    ("There is some ringing in my right ear.",
     {"entities": [(14, 21, "SYMPTOM"), (28, 37, "BODY_PART")]}),
    
    ("You have benign paroxysmal positional vertigo, or BPPV.",
     {"entities": [(9, 45, "DIAGNOSIS"), (50, 54, "DIAGNOSIS")]}),
    
    ("We can fix it with the Epley maneuver.",
     {"entities": [(23, 37, "TREATMENT")]}),
    
    ("I'm prescribing antibiotics for your infection.",
     {"entities": [(16, 27, "TREATMENT")]}),
    
    ("The MRI shows a herniated disc at L4-L5.",
     {"entities": [(16, 30, "DIAGNOSIS"), (34, 39, "BODY_PART")]}),
    
    ("You need to take insulin twice daily for your diabetes.",
     {"entities": [(17, 24, "TREATMENT"), (47, 55, "DIAGNOSIS")]}),
    
    ("The ECG shows atrial fibrillation.",
     {"entities": [(14, 33, "DIAGNOSIS")]}),
    
    ("We'll start you on blood thinners to prevent stroke.",
     {"entities": [(19, 33, "TREATMENT"), (45, 51, "DIAGNOSIS")]}),
    
    ("Patient presents with chronic fatigue and muscle weakness.",
     {"entities": [(22, 37, "SYMPTOM"), (42, 57, "SYMPTOM")]}),
    
    ("The ultrasound revealed gallstones in the gallbladder.",
     {"entities": [(24, 34, "DIAGNOSIS"), (42, 53, "BODY_PART")]}),
    
    ("I'm recommending a colonoscopy to investigate further.",
     {"entities": [(19, 30, "TREATMENT")]}),
    
    ("The patient has been experiencing shortness of breath.",
     {"entities": [(34, 53, "SYMPTOM")]}),
]

print(f"📊 Loaded {len(TRAINING_DATA)} training examples")
print(f"\nSample entry:")
print(f"Text: {TRAINING_DATA[0][0]}")
print(f"Entities: {TRAINING_DATA[0][1]['entities']}")

## 3. Create Training Pipeline

In [ ]:
# Load a blank English model
nlp = spacy.blank("en")

# Create the NER component
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# Add labels to the NER
LABELS = ["SYMPTOM", "TREATMENT", "DIAGNOSIS", "BODY_PART", "PROGNOSIS"]
for label in LABELS:
    ner.add_label(label)

print(f"✅ NER component created with labels: {LABELS}")

In [ ]:
# Prepare training examples
def create_training_examples(training_data, nlp):
    """Convert training data to spaCy Example objects."""
    examples = []
    for text, annotations in training_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        examples.append(example)
    return examples

examples = create_training_examples(TRAINING_DATA, nlp)
print(f"✅ Prepared {len(examples)} training examples")

## 4. Train the Model

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Training configuration
N_ITER = 30  # Number of training iterations
DROPOUT = 0.35  # Dropout rate

# Initialize the model
optimizer = nlp.begin_training()

# Training loop
print("🚀 Starting training...\n")
print(f"{'Iteration':<12} {'Loss':<15}")
print("-" * 30)

losses_history = []

for iteration in range(N_ITER):
    random.shuffle(examples)
    losses = {}
    
    # Batch the examples
    batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
    
    for batch in batches:
        nlp.update(
            batch,
            drop=DROPOUT,
            sgd=optimizer,
            losses=losses
        )
    
    losses_history.append(losses.get("ner", 0))
    
    if (iteration + 1) % 5 == 0 or iteration == 0:
        print(f"{iteration + 1:<12} {losses.get('ner', 0):<15.4f}")

print("\n✅ Training complete!")

In [ ]:
# Visualize training loss
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(range(1, N_ITER + 1), losses_history, 'b-', linewidth=2)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Medical NER Training Loss', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📉 Final loss: {losses_history[-1]:.4f}")
print(f"📈 Loss reduction: {((losses_history[0] - losses_history[-1]) / losses_history[0] * 100):.1f}%")

## 5. Evaluate the Model

In [ ]:
# Test sentences
TEST_SENTENCES = [
    "I've been having severe chest pain and shortness of breath.",
    "The doctor prescribed metformin 500mg twice daily for diabetes.",
    "My knee hurts when I walk and there's swelling around it.",
    "The MRI showed a torn ACL in my left knee.",
    "I'm experiencing fatigue, hair loss, and weight gain.",
]

print("🔍 Testing the trained NER model:\n")
print("=" * 60)

for text in TEST_SENTENCES:
    doc = nlp(text)
    print(f"\nText: {text}")
    
    if doc.ents:
        print("Entities found:")
        for ent in doc.ents:
            print(f"  - {ent.text:<25} [{ent.label_}]")
    else:
        print("  No entities found")
    print("-" * 60)

In [ ]:
# Visualize entities with displaCy
from spacy import displacy

test_text = "Patient presents with severe headache and nausea. Started on ibuprofen 400mg. Diagnosis: migraine with aura."
doc = nlp(test_text)

# Define colors for each entity type
colors = {
    "SYMPTOM": "#ff6b6b",
    "TREATMENT": "#4ecdc4",
    "DIAGNOSIS": "#45b7d1",
    "BODY_PART": "#96ceb4",
    "PROGNOSIS": "#feca57"
}

options = {"colors": colors}

print("Entity Visualization:")
displacy.render(doc, style="ent", options=options, jupyter=True)

## 6. Save the Model

In [ ]:
# Save the trained model
output_dir = Path("medical_ner_model")
output_dir.mkdir(exist_ok=True)

nlp.to_disk(output_dir)
print(f"✅ Model saved to: {output_dir}")

# Verify the model can be loaded
nlp_loaded = spacy.load(output_dir)
print(f"✅ Model loaded successfully!")
print(f"\nPipeline components: {nlp_loaded.pipe_names}")
print(f"NER labels: {nlp_loaded.get_pipe('ner').labels}")

In [ ]:
# Download the model (for Colab)
!zip -r medical_ner_model.zip medical_ner_model/

try:
    from google.colab import files
    files.download('medical_ner_model.zip')
    print("📥 Model downloaded!")
except:
    print("Model saved as medical_ner_model.zip")

## 7. Model Metrics

In [ ]:
from collections import defaultdict

# Calculate precision, recall, F1 on training data
def evaluate_ner(nlp, test_data):
    """Calculate NER metrics."""
    tp = defaultdict(int)  # True positives
    fp = defaultdict(int)  # False positives
    fn = defaultdict(int)  # False negatives
    
    for text, annotations in test_data:
        doc = nlp(text)
        
        # Get predicted entities
        pred_ents = set((ent.start_char, ent.end_char, ent.label_) for ent in doc.ents)
        
        # Get gold entities
        gold_ents = set((ent[0], ent[1], ent[2]) for ent in annotations.get("entities", []))
        
        # Calculate TP, FP, FN
        for ent in pred_ents:
            if ent in gold_ents:
                tp[ent[2]] += 1
            else:
                fp[ent[2]] += 1
        
        for ent in gold_ents:
            if ent not in pred_ents:
                fn[ent[2]] += 1
    
    # Calculate metrics per label
    metrics = {}
    for label in LABELS:
        precision = tp[label] / (tp[label] + fp[label]) if (tp[label] + fp[label]) > 0 else 0
        recall = tp[label] / (tp[label] + fn[label]) if (tp[label] + fn[label]) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        metrics[label] = {"precision": precision, "recall": recall, "f1": f1}
    
    return metrics

metrics = evaluate_ner(nlp, TRAINING_DATA)

print("📊 Model Metrics (on training data):\n")
print(f"{'Label':<15} {'Precision':<12} {'Recall':<12} {'F1 Score':<12}")
print("-" * 51)

for label, scores in metrics.items():
    print(f"{label:<15} {scores['precision']:<12.3f} {scores['recall']:<12.3f} {scores['f1']:<12.3f}")

## Summary

### What We Built
- Custom spaCy NER model for medical text
- 5 entity types: SYMPTOM, TREATMENT, DIAGNOSIS, BODY_PART, PROGNOSIS
- Trained on 50+ labeled medical sentences

### Key Takeaways
1. **Data Quality Matters**: More diverse training examples = better generalization
2. **Iterative Training**: 30 iterations with decreasing loss shows learning
3. **Evaluation**: Track precision/recall per entity type

### Next Steps for Production
1. Expand training data to 1000+ examples
2. Add more entity types (dosage, frequency, severity)
3. Use cross-validation for robust evaluation
4. Fine-tune on domain-specific biomedical models (BioBERT, PubMedBERT)
5. Implement active learning for efficient labeling